In [1]:
# mengimport library yang diperlukan

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
import pandas as pd

data = pd.read_csv('data/insurance.csv')

display(data.head())

display(data.corr())

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


,age,bmi,children,charges
age,1.000000,0.109272,0.042469,0.299008
bmi,0.109272,1.000000,0.012759,0.198341
children,0.042469,0.012759,1.000000,0.067998
charges,0.299008,0.198341,0.067998,1.000000


SOAL 
1. Pastikan semua variabel kategorial diolah dengan baik. (Gunakan fitur mapping pada pandas)

JAWAB
Proses dibawah ini menggunakan fitur mapping untuk mengubah variabel kategorial menjadi angka agar dapat diolah dengan baik

In [3]:
new_labels_sex = {
    'male': 1,
    'female': 0
}

new_labels_smoker ={
    'yes' : 1,
    'no' : 0
}

new_labels_region ={
    'northwest' : 0,
    'southwest' : 1,
    'southeast' : 2,
    'northeast' : 3 
}

# Encode label
data['sex'] = data['sex'].map(new_labels_sex)
data['smoker'] = data['smoker'].map(new_labels_smoker)
data['region'] = data['region'].map(new_labels_region)


# Cek data
data.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,1,16884.92400
1,18,1,33.770,1,0,2,1725.55230
2,28,1,33.000,3,0,2,4449.46200
3,33,1,22.705,0,0,0,21984.47061
4,32,1,28.880,0,0,0,3866.85520


SOAL

2. Cek kondisi multicollinearity untuk semua variabel independen. Jika ada, antar variabel apakah itu?
3. Pastikan model menggunakan variabel yang tidak memiliki nilai multicollinearity yang tinggi
4. (Hints) Anda dapat menggunakan nilai ***Variance Inflation Factor (VIF)*** untuk mengetahui tingkat multicollinearity pada sebuah variabel independent.

JAWAB
- Pada proses dibawah ini dilakukan pengecekan multicollonearity, dengan menggunakan fitur VIF dapat mengetahui tingkat multicollinearity
- setalah dilakukan pengecekan dapat dilihat bahwa yang tidak memiliki nilai multicollinearity yang tinggi yaitu :
    sex = 2.0..
    children = 1.7...
    smoker = 1.2...
    region = 2.7...


In [4]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
  
# the independent variables set
X = data[['age', 'sex' ,'bmi' ,'children', 'smoker', 'region']]
  
# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
  
print(vif_data)

    feature       VIF
0       age  7.588734
1       sex  2.002652
2       bmi  9.938670
3  children  1.799980
4    smoker  1.261195
5    region  2.789478


In [5]:
#Untuk data frame di copy sebelum dilakukan penghapusan pada kolom 'age' dan 'bmi'
data_baru = data.copy()
data_baru.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,1,16884.92400
1,18,1,33.770,1,0,2,1725.55230
2,28,1,33.000,3,0,2,4449.46200
3,33,1,22.705,0,0,0,21984.47061
4,32,1,28.880,0,0,0,3866.85520


In [6]:
#Melakukan penghapusan kolom "age" dan "bmi"
data_baru = data_baru.drop(columns=['age', 'bmi'])
data_baru.head()

,sex,children,smoker,region,charges
0,0,0,1,1,16884.92400
1,1,1,0,2,1725.55230
2,1,3,0,2,4449.46200
3,1,0,0,0,21984.47061
4,1,0,0,0,3866.85520


SOAL

5. Evaluasi model yang Anda buat dengan nilai R2

JAWAB
- Proses kali ini menggunakan fitur r_square untuk menentukan akurasi dari variabel independen

In [7]:
X = data_baru.iloc[:, :-1].values
y = data_baru.iloc[:, 4].values

In [8]:
Xm_train, Xm_test, ym_train, ym_test = train_test_split(X, y, test_size=0.2, random_state=50)

In [9]:
y = y.reshape(len(y), 1)
y.shape

(1338, 1)

In [10]:
mlr = LinearRegression()
mlr.fit(Xm_train, ym_train)

ym_pred = mlr.predict(Xm_test)

In [11]:
gabung = np.concatenate((ym_test, ym_pred))
gabung

array([ 5976.8311    ,  5846.9176    , 13831.1152    ,  9625.92      ,
        2680.9493    , 47896.79135   , 18223.4512    ,  7419.4779    ,
        3732.6251    , 12222.8983    ,  7050.0213    , 21978.6769    ,
        6282.235     , 37701.8768    ,  7046.7222    , 12032.326     ,
       13112.6048    ,  4239.89265   , 12333.828     ,  3410.324     ,
        1727.785     , 44641.1974    , 17128.42608   ,  6112.35295   ,
        4529.477     , 10594.50155   ,  6402.29135   , 46151.1245    ,
        1711.0268    ,  1704.70015   , 45863.205     ,  4687.797     ,
       15019.76005   ,  3180.5101    ,  3861.20965   ,  3443.064     ,
       27117.99378   ,  2709.24395   , 13451.122     , 47928.03      ,
       23563.01618   ,  6710.1919    , 14235.072     , 14001.1338    ,
       27218.43725   , 13390.559     , 41034.2214    ,  2020.177     ,
       14256.1928    , 21232.18226   , 48675.5177    ,  6986.697     ,
        4149.736     , 30942.1918    ,  8023.13545   , 42124.5153    ,
      

In [12]:
r2_mlr = r2_score(ym_test, ym_pred)
print(r2_mlr)

0.6559958079837158


SOAL

6. Simpulkan, variabel independen apa saja yang dapat digunakan untuk menghasilkan model regresi yang baik pada kasus _medical insurance costs_?

In [13]:
#setelah ini dilakukan pengecekan akurasi jika menggunakan semua kolom yang diberikan
X2 = data.iloc[:, :-1].values
y2 = data.iloc[:, 6].values

In [14]:
Xm_train2, Xm_test2, ym_train2, ym_test2 = train_test_split(X2, y2, test_size=0.2, random_state=100)

In [15]:
y2 = y2.reshape(len(y), 1)
y2.shape

(1338, 1)

In [16]:
mlr2 = LinearRegression()
mlr2.fit(Xm_train2, ym_train2)

ym_pred2 = mlr2.predict(Xm_test2)

In [17]:
gabung2 = np.concatenate((ym_test2, ym_pred2))
gabung2

array([ 1826.843     , 20177.67113   ,  7421.19455   ,  1877.9294    ,
       15518.18025   , 29523.1656    , 13143.86485   ,  3279.86855   ,
       18972.495     , 12950.0712    ,  8083.9198    , 18218.16139   ,
        8334.45755   ,  2727.3951    ,  7441.053     , 38711.        ,
       10338.9316    , 11552.904     ,  4751.07      , 11381.3254    ,
       10226.2842    ,  1526.312     ,  7222.78625   , 21259.37795   ,
       10848.1343    , 58571.07448   ,  3268.84665   , 48517.56315   ,
       21880.82      , 13457.9608    , 12592.5345    ,  2457.21115   ,
       47896.79135   , 33732.6867    ,  6600.361     , 14349.8544    ,
       18963.17192   , 39047.285     ,  3645.0894    , 30942.1918    ,
        7624.63      ,  7650.77375   ,  2498.4144    ,  4949.7587    ,
        2196.4732    ,  3972.9247    ,  4415.1588    ,  2362.22905   ,
       11272.33139   ,  5662.225     , 13405.3903    ,  8269.044     ,
        4915.05985   ,  7729.64575   ,  7045.499     , 11674.13      ,
      

In [18]:
r2_mlr2 = r2_score(ym_test2, ym_pred2)
print(r2_mlr2)

0.7936615729239531


KESIMPULAN

Dari kedua hasil yang diberikan yaitu dari pengecekan akurasi 4 kolom variabel indepen dan pengecekan semua kolom dapat disimpulkan bahwa pengecekan akurasi dari 4 kolom variabel independen hasil akurasinya tidak lebih baik dari yang semua kolom